In [2]:
from dm_control import mujoco
# General
import copy
import os
import itertools
from IPython.display import clear_output
import numpy as np

# Graphics-related
import matplotlib
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from IPython.display import HTML
import PIL.Image

def display_video(frames, framerate=30):
    height, width, _ = frames[0].shape
    dpi = 70
    orig_backend = matplotlib.get_backend()
    matplotlib.use('Agg')  # Switch to headless 'Agg' to inhibit figure rendering.
    fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
    matplotlib.use(orig_backend)  # Switch back to the original backend.
    ax.set_axis_off()
    ax.set_aspect('equal')
    ax.set_position([0, 0, 1, 1])
    im = ax.imshow(frames[0])
    def update(frame):
        im.set_data(frame)
        return [im]
    interval = 1000/framerate
    anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                    interval=interval, blit=True, repeat=False)
    return HTML(anim.to_html5_video())
# 设置模型文件路径
model_path = "model/mini_mec_six_arm2.xml"
# 加载模型
physics = mujoco.Physics.from_xml_path(model_path)




# 定义关节控制函数
def control_joints(joint_ids, positions):
    for joint_id, position in zip(joint_ids, positions):
        physics.data.ctrl[joint_id] = position  # 使用 actuator 来控制关节位置

def move(joint_ids, positions):
    for joint_id, position in zip(joint_ids, positions):
        physics.data.ctrl[joint_id] = position  # 使用 actuator 来控制关节位置
# 持续运行模拟
joint_angles = [0, 0, 0, 0, 0, 0]
pos = [10, 10, 10, 10]
duration = 2
frames = []
physics.reset()  # Reset state and time
framerate=30
# Visualize the joint axis
scene_option = mujoco.wrapper.core.MjvOption()

while physics.data.time < duration:
    control_joints([0,1,2,3,4,5], joint_angles)
    physics.step()
    # 每帧关节角度都加0.01
    # print(physics.data.ctrl)
    if len(frames) < physics.data.time * framerate:
        pixels = physics.render(scene_option=scene_option)
        frames.append(pixels)
display_video(frames, framerate)
